# ETL Imagenes - Conversión de imagenes en escala de grises a color usando TensorFlow (Variational Auto Encoder)

In [63]:
import tensorflow as tf
import os
import numpy as np

In [64]:
# Cargar imagenes
COLOR_PATH = 'color/'
GRAY_PATH = 'gray/'
def load_image(filepath):
    is_grey = GRAY_PATH in filepath
    raw_img = tf.io.read_file(filepath)
    img_tensor_int = tf.image.decode_jpeg(raw_img, channels=1 if is_grey else 3)
    rescaled_img = tf.image.resize_with_pad(
        img_tensor_int, 
        64, 
        64, 
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR,
        antialias = False
    )
    img_tensor_int = tf.image.convert_image_dtype(rescaled_img, tf.uint8) #Imagen en enteros de 8 bits
    img_tensor_flt = tf.image.convert_image_dtype(rescaled_img, tf.float32) #Normalización de la imagen
    return img_tensor_int, img_tensor_flt

def save_image(tensor, filepath):
    img_uint8 = tf.image.convert_image_dtype(tensor, tf.uint8)
    image = tf.io.encode_jpeg(img_uint8, format='', quality=95, optimize_size=True)
    tf.io.write_file(filepath, image)

In [65]:
tf.io.gfile.makedirs('gray_processed')
tf.io.gfile.makedirs('color_processed')

for img_path in tf.io.gfile.glob(os.path.join(GRAY_PATH + '*.jpg')):
    img_tensor_int, img_tensor_flt = load_image(img_path)
    save_image(img_tensor_int, os.path.join('gray_processed', os.path.basename(img_path)))

for img_path in tf.io.gfile.glob(os.path.join(COLOR_PATH + '*.jpg')):
    img_tensor_int, img_tensor_flt = load_image(img_path)
    save_image(img_tensor_int, os.path.join('color_processed', os.path.basename(img_path)))